In [1]:
import pandas as pd

from core.fs import FsLoader
from core.repository.maria.conn import maria_home
from utils.timeutil import YearMonth
from utils.timeutil import YearQtr
import numpy as np

begin = YearMonth(2023, 1)
end = YearMonth(2024, 5)

# 월 차트 로드
print("Fetching month chart...")
month_chart = pd.read_sql(
    f"""
    select month_chart.* from month_chart
    where year(date) >= {begin.year}
    """,
    maria_home()
)
print("Month chart loaded.")

# 재무제표 로드
print("Loading financial data...")
fs_loader = FsLoader()
print("Financial data loaded.")


def calc(ym1, ym2):
    print(f"{ym1} {ym2}", end="\r")
    df1 = month_chart[month_chart["년월"] == ym1].set_index("code")
    df1 = df1[df1["date"] == df1["date"].max()]  # date = date.max 아니면, 상장폐지 되는 종목. 매수일이 월말이기 때문에 매수 불가
    df2 = month_chart[month_chart["년월"] == ym2].set_index("code")
    df2["date"] = df2["date"].max()  # date = date.max 아니면, 상장폐지 되는 종목. 정리매매 거치기 때문에 별도 처리 필요 없음.
    df1 = df1[df1["open"] > 0]  # 시가=0 은 거래정지, 따라서 매수불가
    df1 = df1[df1["val_last"] > 1_000_0000]  # 거래량 일정 수준 이상(거래량 적을 시 매수 실패할거라고 가정)
    df2["close"] = df2["close"].astype(float)
    df2.loc[(df2["open"] == 0), "close"] /= 2  # 거래정지 시 투자금이 1/2 된다고 가정

    df = pd.DataFrame({
        "매수년월": ym1,
        "매도년월": ym2,
        "종목명": df1["name"],
        "매수일": df1["date"],
        "매수가": df1["close"],
        "P": df1["cap"],
        "vol": df1["vol"],
        "val": df1["val"],
        "shares": df1["shares"],
        "avg": df1["avg"],
        "전월수익률": df1["close"] / df1["open"] - 1,
        "매도가": df2["close"],
        "매도일": df2["date"],
        "추세1": df1["close"] / df1["avg"],
    }).reindex(df1.index)

    shares = pd.concat([df1["shares"], df2["shares"]], axis=1)
    df = df.loc[shares.max(axis=1) / shares.min(axis=1) < 1.5]  # 증자, 액면분할 제거
    df.loc[df["매도가"].isna(), "매도가"] = 0
    df["수익률"] = df["매도가"] / df["매수가"] - 1
    df["수익률"] = df["수익률"].replace(np.nan, -1)

    settled_qtr = YearQtr.settled_of(ym1.last_date)
    return df.join(fs_loader.load(settled_qtr.year, settled_qtr.qtr))


print("Making historical data...")
month_chart["년월"] = month_chart["date"].apply(YearMonth.from_date)
result = pd.concat([calc(ym, ym.next) for ym in begin.to(end)[:-1]])


def add_factor(name, value):
    factors.append(name)
    result[name] = value


factors = ["P", "val", "수익률", "추세1"]
factors += [col for col in result.columns if col.endswith("QoQ")]
add_factor("EQ/P", result["EQ"] / result["P"])

is_cols = ["R", "GP", "O", "EBT", "E"]
for col in is_cols:
    add_factor(f"{col}/P", result[f"{col}/Y"] / result["P"])
    add_factor(f"{col}/A", result[f"{col}/Y"] / result["A"])  # 자산비율
    add_factor(f"{col}/EQ", result[f"{col}/Y"] / result["EQ"])  # 자본비율
    if col != "R":
        add_factor(f"{col}/R", result[f"{col}/Y"] / result["R/Y"])  # 이익율

result = result.replace([np.inf, -np.inf], np.nan)

for f in factors:
    result[f"{f}_pct"] = (
        result.groupby("매도년월")[f]
        .apply(lambda x: np.ceil(x.rank(pct=True) * 100))
        .reset_index(level=0, drop=True)
    )

result.reset_index(inplace=True)
result.to_csv(".cache/historical_data.csv", index=False)
result

Fetching month chart...
Month chart loaded.
Loading financial data...
Loading fs db...
Financial data loaded.
Making historical data...


,code,매수년월,매도년월,종목명,매수일,매수가,P,vol,val,shares,...,O/EQ_pct,O/R_pct,EBT/P_pct,EBT/A_pct,EBT/EQ_pct,EBT/R_pct,E/P_pct,E/A_pct,E/EQ_pct,E/R_pct
0,000020,2023-01,2023-02,동화약품,2023-01-31,9380.0,2.619972e+11,1730269.0,1.616176e+10,27931470.0,...,62.0,76.0,70.0,70.0,60.0,71.0,69.0,68.0,57.0,69.0
1,000040,2023-01,2023-02,KR모터스,2023-01-31,531.0,5.104952e+10,3343710.0,1.824473e+09,96138465.0,...,10.0,18.0,7.0,15.0,9.0,17.0,7.0,15.0,9.0,17.0
2,000050,2023-01,2023-02,경방,2023-01-31,10700.0,2.933434e+11,135064.0,1.437697e+09,27415270.0,...,55.0,84.0,47.0,35.0,35.0,45.0,41.0,34.0,34.0,39.0
3,000070,2023-01,2023-02,삼양홀딩스,2023-01-31,71900.0,6.157711e+11,153368.0,1.073298e+10,8564271.0,...,53.0,55.0,88.0,49.0,49.0,50.0,88.0,49.0,48.0,50.0
4,000080,2023-01,2023-02,하이트진로,2023-01-31,26300.0,1.844514e+12,3929187.0,9.930246e+10,70133611.0,...,88.0,72.0,59.0,57.0,74.0,59.0,57.0,54.0,72.0,56.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37354,950170,2024-03,2024-04,JTC,2024-03-29,4780.0,2.403796e+11,1351931.0,6.041820e+09,50288623.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37355,950190,2024-03,2024-04,고스트스튜디오,2024-03-29,11050.0,1.500578e+11,360057.0,4.066667e+09,13579892.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37356,950200,2024-03,2024-04,소마젠,2024-03-29,5240.0,1.007969e+11,5698573.0,3.336155e+10,19236053.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37357,950210,2024-03,2024-04,프레스티지바이오파마,2024-03-29,8890.0,5.342548e+11,1225987.0,1.054639e+10,60096155.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
